# Relações e seleção de características

Uma parte importante da análise de dados é compreender as relações existentes entre as características do seu conjunto de dados. A partir dessas relações, podemos eventualmente identificar características redundantes, o que nos permite trabalhar com um conjunto menor de características selecionadas de forma a preservar a parte mais importante da informação existente em nosso conjunto de dados.

## Relações entre características

Os principais tipos de relação entre características que investigamos em nossos dados são covariância, correlação, causalidade e condicionalidade.

### Covariância e correlação

A covariância é um conceito estatístico que avalia o comportamento de uma característica em função de outra. Podemos observar três tipos de correlação:
* **Positiva**: representa uma relação de proporcionalidade direta entre as características. Em outras palavras, o aumento no valor de uma característica implica no aumento no valor da outra característica. 
* **Negativa**: representa uma relação de proporcionalidade inversa entre as características. Em outras palavras, o aumento no valor de uma característica implica na redução do valor da outra característica. 
* **Nula**: representa uma ausência de relação entre as características. Em outras palavras, mudanças em uma característica não produzem um padrão claro de mudança em outra característica.

Em geral, usamos a covariância para analisar a correlação entre um ou mais pares de características. Enquanto a covariância nos informa se existe uma relação entre as características, a correlação quantifica essa relação. Podemos analisar a correlação entre os dados fazendo uso de recursos gráficos e analíticos.

#### Entre um par de características

Podemos analisar a correlação entre um par de características graficamente, através de um **gráfico de dispersão**, ou analiticamente, através de diferentes **métricas de correlação**.

##### Gráfico de dispersão

Nesse tipo de gráfico, os dados são apresentados como pontos em um plano cartesiano, onde cada eixo representa uma característica que se deseja comparar.  A correlação entre as características é avaliada em função do padrão que se apresente. Pra ver um exemplo concreto, vamos usar os dados estimados das medianas da altura e peso do Brasil fornecidas pelo IBGE, divididas por faixas etárias.

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [0]:
peso_altura = pd.read_csv("https://raw.githubusercontent.com/leobezerra/pandas-zero/master/datasets/ibge-peso-altura-brasil.csv", sep=";", decimal=",")
peso_altura.head()

Quando um par de características apresenta covariância **positiva**, observamos uma tendência com inclinação positiva. Esse é o padrão esperado para a relação entre as características `"Altura"` e `"Peso"`, que podemos verificar com o método `regplot` da biblioteca `seaborn`:

In [0]:
sns.regplot(x="Altura", y="Peso", data=peso_altura)
plt.show()

Note que o gráfico acima parece apresentar a mistura de dois padrões. Para a região onde o peso é menor que 65kg, há uma covariância positiva entre as características analisadas (reta de tendência com inclinação positiva):

In [0]:
peso_ate_65 = peso_altura.query("Peso < 65")
sns.regplot(x="Altura", y="Peso", data=peso_ate_65)
plt.show()

Já para a região onde o peso é maior que 65kg, não há padrão formado, o que caracteriza uma **covariância nula** (reta de tendência paralela a um dos eixos):

In [0]:
peso_maior_que_65 = peso_altura.query("Peso > 65")
sns.regplot(x="Altura", y="Peso", data=peso_maior_que_65)
plt.show()

Esse resultado faz sentido quando avaliamos que, em geral, medianas de peso até 65kg representam crianças e adolescentes em idade escolar, onde há uma forte relação entre a altura e o ganho de peso:

In [0]:
peso_ate_65["Idade"].unique()

Já para medianas de peso maiores que 65kg, estamos falando sobre adolescentes em idade universitárias, adultos e idosos, onde não há uma relação direta entre altura e peso:

In [0]:
peso_maior_que_65["Idade"].unique()

Para finalizar, quando um par de características apresenta covariância **negativa**, observamos uma tendência com inclinação negativa. Podemos gerar uma nova característica chamada de `"Faltam para 100kg"` a partir da característica `"Peso"`:

In [0]:
peso_altura["Faltam para 100kg"] = 100 - peso_altura["Peso"]
sns.regplot(x="Faltam para 100kg", y="Altura", data=peso_altura)
plt.show()

Note que a criação dessa nova característica não afetou a existência de dois padrões observada para a característica original. 

##### Métricas de correlação

As métricas de correlação mais comuns são as métricas de **Pearson** e de **Spearman**, que avaliam a aderência dos pontos à tendência observada. 

Para a correlação de Pearson, utilizamos como tendência uma reta, como nos gráfico produzidos com o método `regplot`. Os dados avaliados a seguir apresentam boa aderência à reta de tendência, o que é capturado pela correlação de Pearson como um valor próximo a 1:

In [0]:
print(peso_ate_65[["Altura","Peso"]].corr())
sns.regplot(x=peso_ate_65["Altura"], y=peso_ate_65["Peso"])
plt.show()

Para o caso de correlação negativa, temos que uma correlação forte é avaliada como valores próximos a -1:

In [0]:
peso_ate_65["Faltam para 100kg"] = 100 - peso_ate_65["Peso"]

In [0]:
print(peso_ate_65[["Altura","Faltam para 100kg"]].corr())
sns.regplot(x=peso_ate_65["Altura"], y=peso_ate_65["Faltam para 100kg"])
plt.show()

Por fim, uma correlação fraca será avaliada com valores próximos a 0:

In [0]:
print(peso_maior_que_65[["Altura","Peso"]].corr())
sns.regplot(x=peso_maior_que_65["Altura"], y=peso_maior_que_65["Peso"])
plt.show()

É importante observar que a correlação de Pearson mede apenas a aderência dos dados à tendência, não sendo afetada pela inclinação da reta de tendência (a não ser para indicar a correlação positiva ou negativa). Vamos ver este caso gerando uma característica para o índice de massa corporal:

In [0]:
peso_altura["IMC"] = peso_altura["Peso"] / (peso_altura["Altura"] * peso_altura["Altura"])

In [0]:
print(peso_altura[["Peso","IMC"]].corr())
sns.regplot(x="Peso", y="IMC", data=peso_altura)
plt.show()

Note que, apesar de uma covariância baixa, os dados acima apresentam uma alta correlação. Este exemplo reflete bem a diferença entre a covariância e a correlação, uma vez que a covariância foca na inclinação a reta de tendência, enquanto a correlação foca na aderência dos dados a esta reta:

In [0]:
print(peso_altura[["Peso","IMC"]].cov())

Por medir a aderência dos dados à reta de tendência, a correlação de Pearson é descrita como linear. Em contrapartida, a correlação de Spearman permite avaliar a aderência dos dados a tendências lineares ou não. No exemplo abaixo, vemos que adotar uma medida de tendência não-linear (parâmetro `order=2` do método `regplot`) leva a uma maior aderência dos dados, o que é refletido pela correlação de Spearman:

In [0]:
print(peso_maior_que_65[["Altura","Peso"]].corr(method="spearman"))
sns.regplot(x=peso_maior_que_65["Altura"], y=peso_maior_que_65["Peso"], order=2)
plt.show()

#### Entre múltiplos pares de características

Quando trabalhamos com múltiplos pares de características, primeiro calculamos analiticamente a correlação entre cada par, para em seguida utilizarmos visualizações gráficas.

Como visto acima, o método `corr` presente em objetos `DataFrame` permite o cálculo entre todos os pares de características existentes em um dataframe:

In [0]:
peso_altura.corr()

Para melhorar a visualização dos dados, podemos gerar um **mapa de calor** a partir da matriz de correlações. Neste tipo de gráfico, as cores têm um papel fundamental no entendimento dos dados. Normalmente, cores mais frias (próximas a branco) representam correlações positivas, enquanto cores mais quentes (próximas a preto) nos apontam correlações negativas:

In [0]:
sns.heatmap(peso_altura.corr())
plt.show()

Podemos customizar o método `heatmap` de diferentes formas, trocando por exemplo a legenda lateral por anotações dentro de cada célula do mapa:

In [0]:
sns.heatmap(peso_altura.corr(), annot=True, cbar=False)
plt.show()

### Causalidade e condicionalidade

É importante salientar que uma alta correlação entre duas características **não implica de forma alguma** que uma característica é causada pela outra. De fato, é bastante difícil investigar causalidade apenas a partir da análise de um par de características.

> Esse site mostra fortes correlações entre características, mas que possivelmente não apresentam causalidade entre si https://www.tylervigen.com/spurious-correlations

Uma situação particular é o caso de características condicionais. Neste caso, o dataset disponível só apresenta dados para uma determinada característica quando o dado para uma outra característica satisfaz a determinada condição. 

Vamos analisar essa situação em um dataset sobre preços de casas do Kaggle. Para baixá-lo, siga primeiro [a etapa 1 deste tutorial](https://medium.com/@yvettewu.dw/tutorial-kaggle-api-google-colaboratory-1a054a382de0), que ensina a baixar as credenciais de acesso do Kaggle (`kaggle.json`). Uma vez que você tenha baixado suas credenciais, use o menu ao lado para fazer upload do arquivo para o Colab, e execute as três células abaixo:

In [0]:
!mkdir /root/.kaggle
!cp /content/kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d prevek18/ames-housing-dataset

In [0]:
ames_housing = pd.read_csv("ames-housing-dataset.zip")
ames_housing.head()

> Caso alguma das células acima não funcione, entre em contato com os mantenedores do pandas-zero ;)

Este dataset contém a descrição de várias propriedades, apresentando 82 características sobre cada propriedade. No entanto, quando um conjunto de dados apresenta um número alto de características, é comum que parte dessas características sejam informações adicionais em relação a outra característica.

No caso deste dataset, vamos filtrar apenas as características relacionadas ao termo `"Garagem"`:

In [0]:
ames_garage = ames_housing.filter(like="Garage")
ames_garage.head()

Note que a primeira característica define o tipo de garagem. Esta característica apresenta os seguintes valores:

In [0]:
ames_garage["Garage Type"].value_counts(dropna=False)

Um dos valores possíveis para esta característica é a falta deste dado (`NaN`, que identificamos passando o argumento `dropna=False` ao invocarmos o método `value_counts`).

Quando este dado está em falta, as demais características deixam de fazer sentido:

In [0]:
ames_garage[ames_garage["Garage Type"].isna()]

A forma como procedemos neste tipo de situação depende do contexto. Neste caso, como temos características nominais dependendo desta característica e um baixo número de casos onde a característica original está ausente, a melhor alternativa é remover estas observações:

In [0]:
ames_housing = ames_housing[~ames_housing["Garage Type"].isna()]
ames_housing["Garage Type"].value_counts(dropna=False)

## Seleção de características

A principal vantagem de analisar as relações entre as características presentes em seu conjunto de dados é a possibilidade de selecionar um subconjunto de características que consegue representar a maior parte da informação contida no conjunto original.

Em geral, a **seleção de características** pode ser feita usando critérios analíticos ou com o auxílio de modelos de aprendizado de máquina.

### Critérios analíticos

In [0]:
from sklearn.feature_selection import VarianceThreshold
sel.fit_transform(X)